# Chapter 4

## Figure 4.1, 4.2, and 4.3

Steady states and phase plots in an assymetric network.

In [ ]:
using DifferentialEquations
using LabelledArrays
using UnPack
using Plots
using LinearAlgebra
Plots.default(linewidth=2)

In [ ]:
# Convenience functions
hil(x, k) = x / (x + k)
hil(x, k, n) = hil(x^n, k^n)

In [ ]:
function model41(u, p, t)
    a, b = u
    @unpack k1, k2, k3, k4, k5, n = p
    da = k1 * hil(1, b, n) - (k3 + k5) * a
    db = k2 + k5 * a - k4 * b
    return (da, db)
end

function model41!(D, u, p, t)
    D.a, D.b = model41(u, p, t)
    return nothing
end

In [ ]:
ps1 = LVector(k1=20., k2=5., k3=5., k4=5., k5=2., n=4.)
u0s = [
    LVector(a=0.0, b=0.0),
    LVector(a=0.5, b=0.6),
    LVector(a=0.17, b=1.1),
    LVector(a=0.25, b=1.9),
    LVector(a=1.85, b=1.7),
]

tend = 1.5

In [ ]:
sols = map(u0s) do u0
    prob = ODEProblem(model41!, u0, tend, ps1)
    sol = solve(prob)
end

plot(sols[1], xlabel="Time", ylabel="Concentration", title="Fig. 4.2 A (Time series)", labels=["[A]" "[B]"])

In [ ]:
plot(sols[1], idxs=(:a, :b), xlabel="[A]", ylabel="[B]", aspect_ratio=:equal,
     title="Fig. 4.2 B (Phase plot)", ylims=(0.0, 2.0), xlims=(0.0, 2.0), legend=nothing)

In [ ]:
p43a = plot(title="Fig. 4.3A (Multiple time series)")
	
for sol in sols
    plot!(p43a, sol, linealpha=0.5, legend=nothing)
end

plot!(p43a, xlabel="Time", ylabel="Concentration")

In [ ]:
p43b = plot(title="Fig. 4.3 B (Phase plot)")

for sol in sols
    plot!(p43b, sol, idxs=(:a, :b), legend=nothing)
end

plot!(p43b, xlabel="[A]", ylabel="[B]", xlims=(0., 2.), ylims=(0., 2.), aspect_ratio=:equal)

## Figure 4.4 and 4.5 

Vector fields in phase plots.

In [ ]:
# Mesh points
r = range(0.0, 2.0, 41)
xx = [x for y in r, x in r]
yy = [y for y in r, x in r]

dadb = map(xx, yy) do x, y
    model41((x, y), ps1, 0.0)
end

da = first.(dadb)
db = last.(dadb)

scale = 20 .* (hypot.(da, db).^0.5)
da2 = da ./ scale
db2 = db ./ scale

In [ ]:
plot(title="Fig. 4.4 A (Phase plot with vector field)")

quiver!(
    xx[1:4:end], yy[1:4:end], quiver=(da2[1:4:end], db2[1:4:end]),
    line=(:lightgrey),  arrow=(:closed))

for sol in sols
    plot!(sol, idxs=(:a, :b), linealpha=0.7, legend=nothing)
end

plot!(aspect_ratio=:equal, size=(600, 600), xlims=(r[1], r[end]), ylims=(r[1], r[end]), xlabel="[A]", ylabel="[B]")

In [ ]:
# Figure 4.5A
plot(title="Fig. 4.5 A (Phase plot with nullclines)")

# Phase plots
for sol in sols
    plot!(sol, idxs=(:a, :b), linealpha=0.7, lab=nothing)
end

# nullclines
contour!(r, r, da, levels=[0], cbar=false, line=(:black))
plot!(identity, 0., 0., line=(:black), label="A nullcline")
contour!(r, r, db, levels=[0], cbar=false, line=(:black, :dash))
plot!(identity, 0., 0., line=(:black, :dash), label="B nullcline")
plot!(xlim=(0., 2.), ylim=(0., 2.), legend=:bottomright, size=(600, 600), xlabel="[A]", ylabel="[B]", aspect_ratio=:equal)

In [ ]:
quiver(
    xx[1:4:end], yy[1:4:end], quiver=(da2[1:4:end], db2[1:4:end]),
    line=(:lightgrey), arrow=(:closed),
    xlims=(r[1], r[end]), ylims=(r[1], r[end]))

contour!(r, r, da, levels=[0], cbar=false, line=(:black))
plot!(identity, 0., 0., line=(:black), label="A nullcline")
contour!(r, r, db, levels=[0], cbar=false, line=(:black, :dash))
plot!(identity, 0., 0., line=(:black, :dash), label="B nullcline")
plot!(title="Fig. 4.5 B (Vector field with nullclines)", xlabel="[A]", ylabel="[B]", aspect_ratio=:equal, xlim=(0., 2.), ylim=(0., 2.), legend=:bottomright, size=(600, 600))

### Figure 4.7, 4.8, 4.9, and 4.19A

Symmetric (bistable) biological networks.

In [ ]:
using DifferentialEquations
using LabelledArrays
using UnPack
using Plots
using LinearAlgebra
Plots.default(linewidth=2)

In [ ]:
function model47(u, p, t)
    a, b = u
    @unpack k1, k2, k3, k4, n1, n2 = p
    da = k1 * hil(1, b, n1) - k3 * a
    db = k2 * hil(1, a, n2) - k3 * b
    return (da, db)
end

function model47!(D, u, p, t)
    D.a, D.b = model47(u, p, t)
    return nothing
end

In [ ]:
ps1 = (k1=20., k2=20., k3=5., k4=5., n1=4., n2=1.)
tend = 4.0

sol1 = solve(ODEProblem(model47!, LVector(a=3.,b=1.), tend, ps1))
sol2 = solve(ODEProblem(model47!, LVector(a=1.,b=3.), tend, ps1))

p47a1 = plot(sol1, xlabel="Time", ylabel="Concentration", legend=:right, title= "Fig 4.7A (1)")
p47a2 = plot(sol2, xlabel="Time", ylabel="Concentration", legend=:right, title= "Fig 4.7A (2)")
fig47a = plot(p47a1, p47a2, layout=(2, 1), size=(600, 600))

In [ ]:
r = range(0., 5., 21)
xx = [x for y in r, x in r]
yy = [y for y in r, x in r];

In [ ]:
nca47(b, p) = p.k1 / p.k3 * hil(1, b, p.n1)
ncb47(a, p) = p.k2 / p.k4 * hil(1, a, p.n2)

function ∂F47(a, b, p; scale=20)
    u = LVector(a=a, b=b)
    D = copy(u)
    model47!(D, u, p, nothing)
	return D ./ (norm(D)^0.5 * scale)
end

In [ ]:
p47b = quiver(xx, yy, quiver=(x, y)-> ∂F47(x, y, ps1; scale=20), line=(:lightgrey), arrow=(:closed), aspect_ratio=:equal, size =(600, 600))
plot!(p47b, b -> nca47(b, ps1), identity, 0., 5., label="Nullcline A", line=(:dash, :red))
plot!(p47b, identity, a -> ncb47(a, ps1), 0., 5., label="Nullcline B", line=(:dash, :blue))
plot!(p47b, title="Fig 4.7 B", xlim=(0., 5.), ylim=(0., 5.), aspect_ratio=:equal, size =(600, 600), xlabel="[A]", ylabel="[B]")

In [ ]:
ps2 = LVector(k1=20., k2=20., k3=5., k4=5., n1=4., n2=4.)

tend = 4.0
sol1 = solve(ODEProblem(model47!, LVector(a=3.,b=1.), tend, ps2))
sol2 = solve(ODEProblem(model47!, LVector(a=1.,b=3.), tend, ps2))

pl48a1 = plot(sol1, xlabel="Time", ylabel="Concentration", legend=:right, title= "Fig 4.8A (1)")
pl48a2 = plot(sol2, xlabel="Time", ylabel="Concentration", legend=:right, title= "Fig 4.8A (2)")
fig48a = plot(pl48a1, pl48a2, layout=(2, 1), size=(600, 600))

In [ ]:
p48b = quiver(xx, yy, quiver=(x, y)-> ∂F47(x, y, ps2; scale=20), line=(:lightgrey), arrow=(:closed), aspect_ratio=:equal, size =(600, 600))
plot!(p48b, b -> nca47(b, ps2), identity, 0., 5., label="Nullcline A", line=(:dash, :red))
plot!(p48b, identity, a -> ncb47(a, ps2), 0., 5., label="Nullcline B", line=(:dash, :blue))
plot!(p48b, title="Fig 4.8 B", xlim=(0., 5.), ylim=(0., 5.), aspect_ratio=:equal, size =(600, 600), xlabel="[S1]", ylabel="[S2]")

In [ ]:
r2 = range(1.0, 1.5, 11)
xx2 = [x for y in r2, x in r2]
yy2 = [y for y in r2, x in r2]

p48b = quiver(xx2, yy2, quiver=(x, y)-> ∂F47(x, y, ps2; scale=60), line=(:lightgrey), arrow=(:closed), aspect_ratio=:equal, size =(600, 600))
plot!(p48b, b -> nca47(b, ps2), identity, r2[1], r2[end], label="Nullcline A", line=(:dash, :red))
plot!(p48b, identity, a -> ncb47(a, ps2), r2[1], r2[end], label="Nullcline B", line=(:dash, :blue))
plot!(p48b, title="Fig 4.8 B (close up)", xlim=(r2[1], r2[end]), ylim=(r2[1], r2[end]), aspect_ratio=:equal, size =(600, 600), xlabel="[S1]", ylabel="[S2]")

In [ ]:
pls = map((8.0, 16.0, 20.0, 35.0)) do k1
    ps = LVector(k1=k1, k2=20., k3=5., k4=5., n1=4., n2=4.)
    pl = plot(b -> nca47(b, ps), identity, 0., 7., label="Nullcline A")
    plot!(pl, identity, a -> ncb47(a, ps), 0., 7., label="Nullcline B")
    plot!(pl, title = "K1 = $k1", xlim=(0., 7.), ylim=(0., 7.), aspect_ratio = :equal, xlabel="[A]", ylabel="[B]")
    pl
end

plot(pls..., size = (600, 600))

### Figure 4.11

Surface plots reference: [surface plots @ PlotsGallery.jl](https://goropikari.github.io/PlotsGallery.jl/src/surface.html)

In [ ]:
using Plots
Plots.default(linewidth=2, fmt=:png)

z1(x, y) = x^2 + 0.5y^2
z2(x, y) = (.2x^2-1)^2 + y^2
x1 = y1 = range(-1.0, 1.0, 41)
x2 = range(-2.75, 2.75, 41)
y2 = range(-0.75, 0.75, 41)
p1 = surface(x1, y1, z1, title="Single-well potential")
p2 = contourf(x1, y1, z1)
p3 = surface(x2, y2, z2, title="Double-well potential")
p4 = contourf(x2, y2, z2)

plot(p1, p2, p3, p4, size=(800, 600))

### Figure 4.15, 4.16, and 4.17

Oscillatory networks.

In [ ]:
using DifferentialEquations
using LabelledArrays
using LinearAlgebra
using Plots
Plots.default(linewidth=2)

In [ ]:
function model415!(D, u, p, t)
    @unpack k0, k1, k2, n = p
    @unpack a, b = u
    vab = k1 * a * (1 + b^n)
    D.a = k0 - vab
    D.b = vab - k2 * b
    return nothing
end

In [ ]:
function figure0415(; 
    ps = LVector(k0 = 8., k1 = 1., k2 = 5., n = 2.),
	r = range(0., 4., 21),
	tend = 8.,
	figtitle="Fig 4.15",
    model = model415!,
    u0s = (
        LVector(a=1.5, b=1.0),
        LVector(a=0.0, b=1.0), 
        LVector(a=0.0, b=3.0),
        LVector(a=2.0, b=0.0),
    )
)
	sols = map(u0s) do u0
		solve(ODEProblem(model, u0, tend, ps))
	end

	p1 = plot(sols[1], xlabel="Time", ylabel="Concentration", title ="$figtitle (A)", xlims=(0., 8.))

    function ∂F(x, y; scale=20)
        u = LVector(a=x, b=y)
        D = copy(u)
        model415!(D, u, ps, nothing)
        return D ./ (norm(D)^0.5 * scale)
    end

	nc_a(b) = ps.k0 / ps.k1 * hil(1, b, ps.n)
    nc_b(b) = (ps.k2 * b) / (ps.k1 * (1 + b ^ ps.n))
	
	xx = [x for y in r, x in r]
	yy = [y for y in r, x in r]

	p2 = plot(title = "$figtitle (B)")
	for sol in sols
		plot!(p2, sol, idxs=(:a, :b), label=nothing)
	end
	
	rMin, rMax = r[begin], r[end]
	
	plot!(p2, nc_a, identity, rMin, rMax, label="Nullcline A", line=(:dash, :red))
	plot!(p2, nc_b, identity, rMin, rMax, label="Nullcline B", line=(:dash, :blue))
    quiver!(p2, xx, yy, quiver=∂F, line=(:lightgrey), xlims=(rMin, rMax), ylims=(rMin, rMax), 
        aspect_ratio=:equal, size=(600, 600), arrow=(:closed), xlabel="[A]", ylabel="[B]")
	return (p1, p2)
end

In [ ]:
fig415a, fig415b = figure0415()

In [ ]:
fig415a

In [ ]:
fig415b

In [ ]:
fig416a, fig416b = figure0415(ps = LVector(k0 = 8., k1 = 1., k2 = 5., n = 2.5) , tend = 1000.0, figtitle="Fig 4.16")

In [ ]:
fig416a

In [ ]:
fig416b

In [ ]:
_, fig417 = figure0415(
    ps = LVector(k0 = 8., k1 = 1., k2 = 5., n = 2.5), 
    tend = 10., r = range(0., 4., 21), 
    u0s = ( LVector(a=2.0, b=1.5),),
)

In [ ]:
plot!(fig417, xlims=(1.0, 3.0), ylims=(1.0, 3.0), title="Fig 4.17")

### Figure 4.18 Continuation diagram

See also [BifurcationKit.jl](https://github.com/bifurcationkit/BifurcationKit.jl)

In [ ]:
using DifferentialEquations
using LabelledArrays
using LinearAlgebra
using Plots
Plots.default(linewidth=2)

In [ ]:
function model418!(D, u, p, t)
    @unpack a, b = u
    @unpack k1, k2, k3, k4, k5, n = p
    D.a = k1 * hil(1, b, n) - (k3 + k5) * a
    D.b = k2 + k5 * a - k4 * b
end

In [ ]:
ps = LVector(k1 = 20.0, k2 = 5.0, k3 = 5.0, k4 = 5.0, k5 = 2.0, n = 4)
u0 = LVector(a=0., b=0.)
tend = 1000.

In [ ]:
# Could also use parallel ensemble: https://diffeq.sciml.ai/stable/features/ensemble/

r = range(0., 1000., 51)

aInf = map(r) do k1
    p = copy(ps)
    p[1] = k1
	prob = ODEProblem(model418!, u0, tend, p)
	sol = solve(prob, Rodas5(), save_everystep=false)
	sol.u[end].a
end

plot(r, aInf, title = "Fig 4.18 Continuation diagram", 
     xlabel = "K1" , ylabel= "Steady state [A]", 
     legend=nothing, ylim=(0, 4), xlim=(0, 1000))

### Figure 4.22 Tangent line

In [ ]:
using Plots
Plots.default(linewidth=2)

plot(t -> 3 / (t-2), 2.2, 8.0, lab="Curve")
plot!(t -> 1.5 - (t - 4) * 0.75, 2.7, 5.3, lab="Tangent line")
plot!(title="Fig 4.22", xlabel="Reaction rate", ylabel="Inhibitor concentration", 
      xlims=(2., 8.), ylims=(0., 4.))

## Runtime information

In [ ]:
versioninfo()

In [ ]:
using Pkg
Pkg.status()